Create the layers for the community and non community protected areas

> ```indigenous WDPA filter: desig_eng like '%Aborig%','%Indigen%'
indigenous RAISG filter: use all polygons
non indigenous WDPA filter: desig_eng like '%Commun%','%Conservanc%','%Local%', '%Region%','%Trust%','%Conservator%,'%Private%','%Nature Center%'
Final layer should be the union of the filtered WDPA and RAISG```

First we have to copy the UNEP layer into the EO Wilson organisation, for this we should use the `arcgis` API. 

In [2]:
import arcgis
from arcgis.gis import GIS
import arcpy

In [17]:
import os

In [2]:
import datetime
mydate = datetime.datetime.now()
mydate.strftime("%b")

'Jan'

In [11]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA"

In [3]:
tmp = []
with open(".env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [5]:
unep_wdpa = "ae78aeb913a343d69e950b53e29076f7"

In [6]:
unep_wdpa_item=gis.content.get(unep_wdpa)

Here we need to do some kind of check point making sure we are selecting the `WDPA_poly_Latest` layer. I guess we just need to do an index locator

In [7]:
for lyr in unep_wdpa_item.layers: 
    print(lyr.properties.name)

WDPA_point_Latest
WDPA_poly_Latest


In [8]:
unep_wdpa_url = unep_wdpa_item.layers[1].url

In [9]:
wdpa_comm_name = "wdpa_comm_" + mydate.strftime("%y%b")
wdpa_not_comm_name = "wdpa_not_comm_" + mydate.strftime("%y%b")

In [10]:
workingGDB = r"C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb"

In [27]:
arcpy.conversion.FeatureClassToFeatureClass(in_features = unep_wdpa_url, 
                                            out_path = workingGDB,
                                            out_name = wdpa_comm_name,
                                            where_clause = "DESIG_ENG LIKE  '%Aborig%' or DESIG_ENG LIKE '%Indigen%'  or DESIG_ENG LIKE '%Commun%' or DESIG_ENG LIKE '%Conservanc%' or DESIG_ENG LIKE '%Local%' or DESIG_ENG LIKE '%Region%' or DESIG_ENG LIKE '%Trust%' or DESIG_ENG LIKE '%Conservator%' or DESIG_ENG LIKE '%Private%' or DESIG_ENG LIKE '%Nature Center%'")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\tryArcpy'>

In [28]:
arcpy.conversion.FeatureClassToFeatureClass(in_features = unep_wdpa_url, 
                                            out_path = workingGDB,
                                            out_name = wdpa_not_comm_name,
                                            where_clause = "DESIG_ENG NOT LIKE  '%Aborig%' and DESIG_ENG NOT LIKE '%Indigen%'  and DESIG_ENG NOT LIKE '%Commun%' and DESIG_ENG NOT LIKE '%Conservanc%' and DESIG_ENG NOT LIKE '%Local%' and DESIG_ENG NOT LIKE '%Region%' and DESIG_ENG NOT LIKE '%Trust%' and DESIG_ENG NOT LIKE '%Conservator%' and DESIG_ENG NOT LIKE '%Private%' and DESIG_ENG NOT LIKE '%Nature Center%'")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\wdpa_not_comm'>

To get the inverted selection the where clause should be `not like` and instead of `or` it should be `and` DONE

once the two wdpa bits are done we have to merge the RAISG layer, it can be living in the AOL account. Although the original data lives also here https://geo.socioambiental.org/webadaptor2/rest/services/raisg/raisg_tis/MapServer/2 but this is a map server, not sure what are the permissions to work with it. 

In [11]:
raisg = "f3cd7abb0f014fb09542d8553eba4b25"
raisg_item=gis.content.get(raisg)
for lyr in raisg_item.layers: 
    print(lyr.properties.name)

Tis_ComunIndigenas
Tis_TerritoriosIndigenas


Creating and calculating the `NAME` and `REP_AREA` fields for the RAISG layer has been done in arcgis Online. 

In [12]:
raisg_item.layers[1].properties.fields

[{
   "name": "FID",
   "type": "esriFieldTypeOID",
   "actualType": "int",
   "alias": "FID",
   "sqlType": "sqlTypeInteger",
   "nullable": false,
   "editable": false,
   "domain": null,
   "defaultValue": null
 }, {
   "name": "pais",
   "type": "esriFieldTypeString",
   "actualType": "nvarchar",
   "alias": "pais",
   "sqlType": "sqlTypeNVarchar",
   "length": 20,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 }, {
   "name": "categoria",
   "type": "esriFieldTypeString",
   "actualType": "nvarchar",
   "alias": "categoria",
   "sqlType": "sqlTypeNVarchar",
   "length": 60,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 }, {
   "name": "nombre",
   "type": "esriFieldTypeString",
   "actualType": "nvarchar",
   "alias": "nombre",
   "sqlType": "sqlTypeNVarchar",
   "length": 100,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 }, {
   "name": "status",
   "type": "esr

In [13]:
tis_raisg_url = raisg_item.layers[1].url

Currently the app uses the fields from the wdpa layers to feed the tooltips, the fields are `NAME` and `REP_AREA`. The latter is the area in km^2. 
Meanwhile the TIs layer has information on the name, which has 85 missing data, and the area in ha. The fields are: `nombre` and `area_sig_h` 
So we have to calculate the RAISG area in km2 and create a new field with the name changing the field's name. 

In [14]:
#wdpa_comm_path = workingGDB + r"\\" + wdpa_comm_name
#out_merge = workingGDB + r"\\" + "wdpa_raisg_merge_" + mydate.strftime("%y%b")

In [34]:
wdpa_comm_path = os.path.join(workingGDB, wdpa_comm_name)
out_merge = os.path.join(workingGDB,"wdpa_raisg_merge_" + mydate.strftime("%y%b"))

In [37]:
print(wdpa_comm_path)

C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\wdpa_comm_20Jan


In [15]:
# Create FieldMappings object to manage merge output fields
fieldMappings = arcpy.FieldMappings()

# Add all fields from both TIs and the community WDPA
fieldMappings.addTable(tis_raisg_url)
fieldMappings.addTable(wdpa_comm_path)

In [16]:
# Remove the fields NAME and REP_AREA from the field map (we are going to map how they have to be merged)
for field in fieldMappings.fields:
    if field.name in ["NAME", "REP_AREA"]:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

In [17]:
# Add input fields "NAME" from both layers into new output field 
fldMap_protName = arcpy.FieldMap()
fldMap_protName.addInputField(wdpa_comm_path, "NAME")
fldMap_protName.addInputField(tis_raisg_url, "NAME")

# Set name of new output field "NAME"
protName = fldMap_protName.outputField
protName.name = "NAME"
fldMap_protName.outputField = protName

# Add output field to field mappings object
fieldMappings.addFieldMap(fldMap_protName)

In [20]:
# Add input fields "REP_AREA" from both layers into new output field 
fldMap_protArea = arcpy.FieldMap()
fldMap_protArea.addInputField(wdpa_comm_path, "REP_AREA")
fldMap_protArea.addInputField(tis_raisg_url, "REP_AREA")

# Set name of new output field "REP_AREA"
protArea = fldMap_protArea.outputField
protArea.name = "REP_AREA"
fldMap_protArea.outputField = protArea

# Add output field to field mappings object
fieldMappings.addFieldMap(fldMap_protArea)

In [21]:
arcpy.management.Merge([tis_raisg_url, wdpa_comm_path], 
                       out_merge, 
                      fieldMappings)

<Result 'C:\\\\Users\\\\Vizuser\\\\Documents\\\\ArcGIS\\\\Projects\\\\WDPA\\\\WDPA.gdb\\wdpa_raisg_merge_20Jan'>

From here we are going to try to publish the data to arcgis online. 

In [21]:
p = arcpy.mp.ArcGISProject("C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA_jan.aprx")

In [22]:
for m in p.listMaps():
    print(m.name)

Map
community-based
Protected areas


In [23]:
pa_map = p.listMaps()[2]
cb_map = p.listMaps()[1]

In [33]:
for lyr in cb_map.listLayers():
    print(lyr.name)

wdpa_raisg_merge_20Jan


In [34]:
output_path = os.path.join("packages", cb_map.name + '.vtpk')
output_path

'packages\\community-based.vtpk'

In [35]:
sharing_draft = cb_map.getWebLayerSharingDraft("HOSTING_SERVER", "TILE", "TileSharingDraftExample")

In [36]:
sharing_draft.summary = "My Summary"
sharing_draft.tags = "My Tags"
sharing_draft.description = "My Description"
sharing_draft.credits = "My Credits"
sharing_draft.useLimitations = "My Use Limitations"

In [38]:
sharing_draft.exportToSDDraft("..\\packages\\TileSharingDraftExample.sddraft")

In [40]:
sd_output_filename

'..\\packages\\TileSharingDraftExample.sd'

In [41]:
# Stage Service
sd_filename = "TileSharingDraftExample" + ".sd"
sd_output_filename = os.path.join("..\\packages", sd_filename)
arcpy.StageService_server("..\\packages\\TileSharingDraftExample.sddraft", "..\\packages\\TileSharingDraftExample.sd")

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (StageService).


In [20]:
 arcpy.management.CreateVectorTilePackage(pa_map, 
                                          r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\packages\Protected_areas.vtpk", 
                                          "ONLINE", None, "INDEXED", 295828763.795777, 72223.819286, None, '', '')

ExecuteError: ERROR 001852: Layer type is not supported for vector tile packages.
Failed to execute (CreateVectorTilePackage).


In [19]:
output_path = os.path.join("packages", pa_map.name + '.vtpk')
arcpy.CreateVectorTilePackage_management(pa_map, 
                                         output_path, 
                                         "ONLINE", None, "INDEXED", 
                                         295828763.795777, 72223.819286, None, '', '')

ExecuteError: ERROR 001852: Layer type is not supported for vector tile packages.
Failed to execute (CreateVectorTilePackage).
